In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
from functools import reduce

## Stock Feature Selection: Adj Close, Volume, RSI, ATR

In [35]:
tickers = pd.read_csv('tickers.csv')
tickers.sort_values(by='Market Cap', ascending=False, inplace=True)
top18_tickers = tickers[:20]
top18_tickers.reset_index(inplace=True)
top18_tickers = top18_tickers.drop([7,8])
stocks = top18_tickers['Symbol'].to_list()
start_date = '2013-01-01'
end_date = '2019-01-01'
data = yf.download("AAPL", start=start_date, end=end_date)
data = data.reset_index()
dates = data['Date']

[*********************100%%**********************]  1 of 1 completed


In [44]:
def get_log_returns(list_of_stocks_tickers, start_date, end_date, interval='1d'):
    stocks = list()
    for ticker in list_of_stocks_tickers:
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        data = {ticker: np.log(data['Adj Close']) - np.log(data['Adj Close'].shift(1))}
        log_return = pd.DataFrame(data=data)
        stocks.append(log_return)
    all_stocks = reduce(lambda df1, df2: pd.merge(df1, df2, on='Date'), stocks)
    all_stocks['Date'] = log_return.index
    return all_stocks

In [45]:
stocks

['AAPL',
 'MSFT',
 'GOOG',
 'GOOGL',
 'AMZN',
 'NVDA',
 'TSLA',
 'META',
 'HSBC',
 'LLY',
 'TSM',
 'V',
 'UNH',
 'XOM',
 'JPM',
 'WMT',
 'NVO',
 'JNJ']

In [46]:
returns = get_log_returns(stocks, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [63]:
returns.to_csv("base_returns.csv")

In [48]:
def calculate_rsi(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1]  # The diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

In [49]:
def calculate_rsi(data, period=14):
    # Calculate price differences
    price_diff = np.diff(data)
    
    # Calculate initial seed values
    seed = price_diff[:period + 1]
    positive_seed = seed[seed >= 0].sum()
    negative_seed = -seed[seed < 0].sum()
    
    # Calculate initial RS and RSI
    initial_rs = positive_seed / negative_seed
    initial_rsi = 100 - 100 / (1 + initial_rs)
    
    # Initialize up and down values
    up = positive_seed / period
    down = negative_seed / period
    
    # Initialize RSI array
    rsi = np.zeros_like(data)
    rsi[:period] = initial_rsi

    for i in range(period, len(data)):
        # Calculate delta for the current period
        delta = price_diff[i - 1]
        
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        
        # Update up and down values
        up = (up * (period - 1) + upval) / period
        down = (down * (period - 1) + downval) / period
        
        # Calculate RS and RSI for the current period
        current_rs = up / down
        rsi[i] = 100 - 100 / (1 + current_rs)

    return rsi


In [50]:
def calculate_atr(data, period=14):
    data = np.array(data)
    
    high = data[:, 0]
    low = data[:, 1]
    close = data[:, 2]
    
    tr = np.maximum(high - low, np.abs(high - np.roll(close, 1)), np.abs(low - np.roll(close, 1)))
    
    atr = np.zeros_like(tr)
    atr[period] = np.mean(tr[:period])
    
    for i in range(period + 1, len(tr)):
        atr[i] = ((period - 1) * atr[i - 1] + tr[i]) / period
    
    return atr

## Model Param

In [51]:
params = {
    "BATCH_SIZE": 50,
    "EPOCHS": 10,
    "LR": 0.00010000,
    "TIME_STEPS": 60
    }

TIME_STEPS = params['TIME_STEPS']
BATCH_SIZE = params['BATCH_SIZE']


def build_timeseries(mat, y_col_index):
    
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]

    x = np.zeros((dim_0, TIME_STEPS, dim_1))
    y = np.zeros((dim_0,))

    print("Length of inputs", dim_0)

    for i in range(dim_0):
        x[i] = mat[i:TIME_STEPS+i]
        y[i] = mat[TIME_STEPS+i, y_col_index]

    print("length of time-series - inputs", x.shape)
    print("length of time-series - outputs", y.shape)


    return x, y
     

## Model

In [52]:
pip install keras_tuner

Note: you may need to restart the kernel to use updated packages.


In [53]:
from sklearn.preprocessing import MinMaxScaler
import keras
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, Activation,concatenate, Attention, Bidirectional,GlobalAveragePooling1D
from keras import optimizers
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import LeakyReLU
import matplotlib.pyplot as plt
import keras_tuner as kt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [57]:
def model_builder(hyperparameters):
    lstm_model = Sequential()
    
    hyperparameters_units = hyperparameters.Int('units', min_value=50, max_value=100, step=TIME_STEPS)
    lstm_model.add(LSTM(units=hyperparameters_units, 
                        input_shape=(x_t.shape[1], x_t.shape[2]), 
                        return_sequences=True,
                        kernel_initializer='he_normal'))
    lstm_model.add(GlobalAveragePooling1D())
    lstm_model.add(Dense(60,activation='relu'))
    lstm_model.add(Dense(20,activation='relu'))
    lstm_model.add(Dropout(0.05))
    lstm_model.add(Dense(1, activation='linear'))
    
    hyperparameters_learning_rate = hyperparameters.Choice('learning_rate', values=[0.01, 0.05, 0.1])
    
    lstm_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=hyperparameters_learning_rate))
    
    return lstm_model

In [60]:
mse = {}
total_predicted_returns = pd.DataFrame()
for ticker in stocks:
    df = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    df['RSI'] = calculate_rsi(df['Adj Close'])
    df['ATR'] = calculate_atr(df[['High', 'Low', 'Close']])
    stock_returns = returns[[ticker]].reset_index()
    stock_returns['Date'] = stock_returns['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df.reset_index(inplace=True)
    df_stock = df[['RSI', 'ATR', 'Date', 'Volume']]
    stock_returns.Date = pd.to_datetime(stock_returns.Date)
    df_stock.Date = pd.to_datetime(df_stock.Date)
    data = pd.merge(df_stock[['RSI', 'ATR', 'Date', 'Volume']],  stock_returns , on='Date')[1:].rename(columns={ticker : 'Returns'})
    print(data)
    # Split the data we try by years first
    data = data[data['Date'] < '2019-01-01']
    df_train = data[data['Date'] < '2018-01-01']
    predicted_period = data[(data['Date'] >= '2018-01-01') & (data['Date'] < '2019-01-01')][['Date']]
    print(len(data) - len(predicted_period) - 60)
    df_test = data[len(data) - len(predicted_period) - 60:]
    train_cols = ["RSI", "ATR", "Volume", "Returns"]
    x = df_train[train_cols].values
    print(x)
    
    #scaling
    min_max_scaler = MinMaxScaler(feature_range = (0, 1))
    x_train = min_max_scaler.fit_transform(x)
    x_test = min_max_scaler.transform(df_test[train_cols])
    
    x_t, y_t = build_timeseries(x_train, 1)
    print("Training Size", x_t.shape, y_t.shape)
    
    x_t_test, y_t_test = build_timeseries(x_test, 1)
    print("Test Size", x_t_test.shape, y_t_test.shape)
    
    x_left, x_val = train_test_split(x_t, test_size=0.2, shuffle=False)
    y_left, y_val = train_test_split(y_t, test_size=0.2, shuffle=False)
    
    tuner = kt.BayesianOptimization(
        model_builder,
        objective='val_loss',
        max_trials=5)
    tuner.search(x_t, y_t, epochs=5, validation_data=(x_val,y_val))
    lstm_model  = tuner.get_best_models()[0]  
    print(ticker,lstm_model.summary())
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    #Model Training
    history_lstm = lstm_model.fit(x_t, y_t, epochs=params["EPOCHS"], verbose=1, batch_size=BATCH_SIZE, callbacks=[callback],
                                shuffle=False)
    y_pred_lstm = lstm_model.predict(x_t_test, batch_size=BATCH_SIZE)
    print(y_pred_lstm)
    y_pred_lstm = y_pred_lstm.flatten()
    print(y_pred_lstm)
    error_lstm = mean_squared_error(y_t_test, y_pred_lstm)
    mse[ticker] = error_lstm
    print(ticker,error_lstm)
    
    y_pred_lstm_org = (y_pred_lstm * min_max_scaler.data_range_[1]) + min_max_scaler.data_min_[1]   #Inverse Transform 
    predicted_returns = pd.Series(y_pred_lstm_org).to_frame(ticker)
    predicted_period = predicted_period.reset_index().drop(columns=['index'])
    result = predicted_returns.join(predicted_period)
    if  total_predicted_returns.empty:
        total_predicted_returns= result 
    else: 
        total_predicted_returns = pd.merge(result, total_predicted_returns, on='Date')

[*********************100%%**********************]  1 of 1 completed
            RSI       ATR       Date     Volume   Returns
1     23.009684  0.000000 2013-01-03  352965200 -0.012703
2     23.009684  0.000000 2013-01-04  594333600 -0.028250
3     23.009684  0.000000 2013-01-07  484156400 -0.005900
4     23.009684  0.000000 2013-01-08  458707200  0.002688
5     23.009684  0.000000 2013-01-09  407604400 -0.015752
...         ...       ...        ...        ...       ...
1505  22.983592  1.395621 2018-12-24  148676800 -0.026215
1506  36.492563  1.483613 2018-12-26  234330000  0.068053
1507  35.825001  1.497283 2018-12-27  212468400 -0.006511
1508  35.924049  1.461227 2018-12-28  169165600  0.000512
1509  37.872277  1.412747 2018-12-31  140014000  0.009619

[1509 rows x 5 columns]
1198
[[ 2.30096839e+01  0.00000000e+00  3.52965200e+08 -1.27027104e-02]
 [ 2.30096839e+01  0.00000000e+00  5.94333600e+08 -2.82499616e-02]
 [ 2.30096839e+01  0.00000000e+00  4.84156400e+08 -5.89961289e-03]
 ...

C:\Users\user\AppData\Local\Temp\ipykernel_23460\2841010685.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.Date = pd.to_datetime(df_stock.Date)
C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 452)           826256    
                                                                 
 global_average_pooling1d (  (None, 452)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 60)                27180     
                                                                 
 dense_1 (Dense)             (None, 20)                1220      
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                        

[*********************100%%**********************]  1 of 1 completed
            RSI       ATR       Date    Volume   Returns
1     50.157734  0.000000 2013-01-03  48294400 -0.013487
2     50.157734  0.000000 2013-01-04  52521100 -0.018893
3     50.157734  0.000000 2013-01-07  37110400 -0.001872
4     50.157734  0.000000 2013-01-08  44703100 -0.005259
5     50.157734  0.000000 2013-01-09  49047900  0.005634
...         ...       ...        ...       ...       ...
1505  29.476164  3.722587 2018-12-24  43935200 -0.042635
1506  43.112583  3.937402 2018-12-26  51634800  0.066078
1507  44.232346  3.998302 2018-12-27  49498500  0.006146
1508  43.069046  3.919138 2018-12-28  38196300 -0.007838
1509  45.379688  3.782771 2018-12-31  33173800  0.011685

[1509 rows x 5 columns]
1198
[[ 5.01577338e+01  0.00000000e+00  4.82944000e+07 -1.34866373e-02]
 [ 5.01577338e+01  0.00000000e+00  5.25211000e+07 -1.88929878e-02]
 [ 5.01577338e+01  0.00000000e+00  3.71104000e+07 -1.87150602e-03]
 ...
 [ 5.882161

C:\Users\user\AppData\Local\Temp\ipykernel_23460\2841010685.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.Date = pd.to_datetime(df_stock.Date)
C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


...

C:\Users\user\AppData\Local\Temp\ipykernel_23460\2841010685.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.Date = pd.to_datetime(df_stock.Date)
C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 392)           622496    
                                                                 
 global_average_pooling1d (  (None, 392)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 60)                23580     
                                                                 
 dense_1 (Dense)             (None, 20)                1220      
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                        

[*********************100%%**********************]  1 of 1 completed
            RSI       ATR       Date    Volume   Returns
1     76.776775  0.000000 2013-01-03   9598300 -0.001413
2     76.776775  0.000000 2013-01-04  11631800  0.011386
3     76.776775  0.000000 2013-01-07   7548800 -0.002098
4     76.776775  0.000000 2013-01-08   9825300  0.000140
5     76.776775  0.000000 2013-01-09   7672800  0.004471
...         ...       ...        ...       ...       ...
1505  22.357813  3.818837 2018-12-24   7531900 -0.041851
1506  32.109627  3.885349 2018-12-26   9253000  0.031018
1507  33.730988  3.917824 2018-12-27   9918700  0.005509
1508  33.558369  3.780837 2018-12-28   6537200 -0.001100
1509  37.909004  3.673635 2018-12-31   7409900  0.013889

[1509 rows x 5 columns]
1198
[[ 7.67767746e+01  0.00000000e+00  9.59830000e+06 -1.41273000e-03]
 [ 7.67767746e+01  0.00000000e+00  1.16318000e+07  1.13855158e-02]
 [ 7.67767746e+01  0.00000000e+00  7.54880000e+06 -2.09842788e-03]
 ...
 [ 5.171528

C:\Users\user\AppData\Local\Temp\ipykernel_23460\2841010685.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock.Date = pd.to_datetime(df_stock.Date)
C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 332)           447536    
                                                                 
 global_average_pooling1d (  (None, 332)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 60)                19980     
                                                                 
 dense_1 (Dense)             (None, 20)                1220      
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                        

In [62]:
total_predicted_returns.to_csv("LSTM_total_predicted_returns.csv")

In [64]:
total_predicted_returns

,JNJ,Date,NVO,WMT,JPM,XOM,UNH,V,TSM,LLY,HSBC,META,TSLA,NVDA,AMZN,GOOGL,GOOG,MSFT,AAPL
0,1.187479,2018-01-02,0.400482,0.851292,1.276625,0.621484,2.294138,1.294258,0.405146,1.450552,0.481816,1.822085,0.585421,0.387131,1.116622,0.633493,0.511192,1.013067,0.505889
1,1.187405,2018-01-03,0.400483,0.850694,1.275808,0.624629,2.294166,1.294105,0.405186,1.449007,0.481994,1.822085,0.583733,0.387053,1.115542,0.631531,0.512316,1.012728,0.506011
2,1.187378,2018-01-04,0.400508,0.851775,1.275522,0.626881,2.294266,1.294041,0.405329,1.448271,0.482068,1.822085,0.582224,0.387071,1.115245,0.629246,0.510768,1.012687,0.506570
3,1.187370,2018-01-05,0.400309,0.853269,1.276259,0.630076,2.294626,1.293972,0.405153,1.442085,0.481276,1.822085,0.581937,0.387092,1.114246,0.628038,0.506848,1.012696,0.506658
4,1.187713,2018-01-08,0.400484,0.854592,1.275959,0.633020,2.295369,1.294471,0.405125,1.441783,0.481733,1.822085,0.580866,0.386907,1.116062,0.629064,0.510714,1.012563,0.507224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,1.187132,2018-12-24,0.400016,0.844945,1.273794,0.922194,2.328972,1.293092,0.404957,1.443397,0.481059,1.822085,0.720744,0.390519,1.114677,0.684076,0.487627,1.012886,0.507017
247,1.187102,2018-12-26,0.399865,0.844872,1.272514,0.928911,2.330931,1.292784,0.404556,1.445079,0.480850,1.822085,0.719267,0.390622,1.111705,0.685611,0.483614,1.012709,0.507593
248,1.187556,2018-12-27,0.399684,0.845412,1.273991,0.934888,2.332260,1.293167,0.405026,1.445903,0.480990,1.822085,0.724933,0.390542,1.113112,0.679355,0.485190,1.012827,0.508079
249,1.187314,2018-12-28,0.399941,0.845919,1.274330,0.942548,2.333333,1.292291,0.404412,1.446424,0.480522,1.822085,0.721980,0.390401,1.109814,0.679753,0.485605,1.012765,0.508645
